import dependencies

In [1]:
import openai
import instructor 
from qdrant_client import QdrantClient
from pydantic import BaseModel, Field




Mock Example

In [2]:
prompt = """
You are a helpful assistant.
Return an answer to the question.
Question: What is your name?
"""

In [3]:
response = openai.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[{"role": "system", "content": prompt}],
    temperature = 0.0
)
print(response.choices[0].message.content)


I am ChatGPT, your AI assistant. How can I help you today?


Add Instructor(Structured Outputs)

In [5]:
class RAGGenerationResponse(BaseModel):
    answer: str = Field(description="The answer to the question")
    #sources: list[str] = Field(description="The sources used to answer the question, should be a list of urls")



In [4]:
client = instructor.from_openai(openai.OpenAI())

In [6]:
response, raw_response= client.chat.completions.create_with_completion(
    model="gpt-4.1-mini",
    messages=[{"role": "system", "content": prompt}],
    temperature = 0.0,
    response_model=RAGGenerationResponse
)


In [7]:
response

RAGGenerationResponse(answer='My name is ChatGPT. How can I assist you today?')

In [8]:
raw_response

ChatCompletion(id='chatcmpl-D5PdBtb7VAoEuqzKf3G9pO4Xf1dOs', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_O45gzDatwwPmIjR3rLOWYoWU', function=Function(arguments='{"answer":"My name is ChatGPT. How can I assist you today?"}', name='RAGGenerationResponse'), type='function')]))], created=1770182537, model='gpt-4.1-mini-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_8d5bcdc9a1', usage=CompletionUsage(completion_tokens=17, prompt_tokens=95, total_tokens=112, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=None, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=None), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

RAG Example

In [9]:
class RAGGenerationResponse(BaseModel):
    answer: str = Field(description="The answer to the question")
    #sources: list[str] = Field(description="The sources used to answer the question, should be a list of urls")

In [12]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )

    return response.data[0].embedding


def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-01",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }


def process_context(context):

    formatted_context = ""

    for id, chunk, rating in zip(context["retrieved_context_ids"], context["retrieved_context"], context["retrieved_context_ratings"]):
        formatted_context += f"- ID: {id}, rating: {rating}, description: {chunk}\n"

    return formatted_context


def build_prompt(preprocessed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt


def generate_answer(prompt):

    response, raw_response = client.chat.completions.create_with_completion(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}],
        temperature = 0.0,
        response_model=RAGGenerationResponse
    )

    return response


def rag_pipeline(question, qdrant_client, top_k=5):

    

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    final_result = {
        "data_model": answer,
        "answer": answer.answer,
        "question": question,
        "retrieved_context_ids": retrieved_context["retrieved_context_ids"],
        "retrieved_context": retrieved_context["retrieved_context"],
        "similarity_scores": retrieved_context["similarity_scores"]
    }

    return final_result

In [13]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [14]:
output = rag_pipeline("What is the most popular product?", qdrant_client)

In [15]:
output

{'data_model': RAGGenerationResponse(answer='The most popular product based on the provided ratings is the 1TB USB Flash Drive (ID: B0CF57H28T) with a rating of 4.8.'),
 'answer': 'The most popular product based on the provided ratings is the 1TB USB Flash Drive (ID: B0CF57H28T) with a rating of 4.8.',
 'question': 'What is the most popular product?',
 'retrieved_context_ids': ['B0CH6P8DYF',
  'B0BFPZGYLD',
  'B0CF57H28T',
  'B09LVX3XW2',
  'B0BP9Z159S'],
 'retrieved_context': ['New 2023 80x100 High Powered Monoculars for Adults high Powered BAK-4 Prism and FMC Lens Monocular Telescope for Smartphone Monoculars for Bird Watching/Wildlife/Hunting/Hiking 【80x100 High Power Monocular】100 mm object diameter monocular, 80 times magnification. Using optical technology, this monocular uses a fully coated lens to ensure excellent light transmittance and image brightness, provide images with high definition, high quality and high color saturation. 【Compact Monocular】Compact and light monocular 